In [1]:
import cv2
import numpy as np
from keras.models import load_model
import warnings

warnings.filterwarnings('ignore')

# Load the pre-trained model
model = load_model('best_model.h5')

# Define the class labels corresponding to your gestures
label = ['0', '1', '2', '3', '4', '5', 'Hang_Loose', 'L-shaped_Hand', 'Punch', 'Thumbs_Horizontal', 'Thumbs_Up']

# Initialize the video capture from webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Variables for background averaging
aweight = 0.5
num_frames = 0
bg = None

# Function to calculate the running average of the background
def run_avg(img, aweight):
    global bg
    if bg is None:
        bg = img.copy().astype("float")
        return
    cv2.accumulateWeighted(img, bg, aweight)

# Function to segment the hand region in the frame
def segment(img, thres=25):
    global bg
    diff = cv2.absdiff(bg.astype('uint8'), img)
    _, thresholded = cv2.threshold(diff, thres, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        # Use contour with the largest area (assuming it's the hand)
        segmented = max(contours, key=cv2.contourArea)
        
        # Contour approximation to reduce points in the contour
        epsilon = 0.01 * cv2.arcLength(segmented, True)
        approx_contour = cv2.approxPolyDP(segmented, epsilon, True)
        
        return thresholded, approx_contour

# Main loop for real-time gesture recognition
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        # Flip the frame horizontally for a mirror effect
        frame = cv2.flip(frame, 1)
        clone = frame.copy()

        # Define region of interest (ROI) where hand gesture will be detected
        roi = frame[100:300, 300:500]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # Run background averaging for the first few frames to calculate the background
        if num_frames < 30:
            run_avg(gray, aweight)
        else:
            # Segment the hand from the background
            hand = segment(gray)

            if hand is not None:
                (thresholded, segmented) = hand
                cv2.drawContours(clone, [segmented + (300, 100)], -1, (0, 0, 255))

                # Display the thresholded hand
                cv2.imshow("Thresholded", thresholded)

                # Filter the contours based on area to detect hand gesture
                if cv2.contourArea(segmented) > 5000:  # Adjust the threshold for minimum area
                    print("Hand detected for prediction")

                    # Predict the gesture using the model (without normalization)
                    resized_frame = cv2.resize(thresholded, (50, 50))
                    reshaped_frame = np.reshape(resized_frame, (1, 50, 50, 1))
                    predictions = model.predict(reshaped_frame)
                    predicted_class = np.argmax(predictions)

                    # Display the predicted gesture on the screen
                    gesture_name = label[predicted_class]
                    cv2.putText(clone, f'Gesture: {gesture_name}', (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Draw the region of interest (ROI) on the frame
        cv2.rectangle(clone, (300, 100), (500, 300), (0, 255, 0), 2)
        cv2.putText(clone, "Gesture Controlled System", (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 30, 255), 3)

        # Increment the number of frames
        num_frames += 1

        # Show the video frame
        cv2.imshow('Gesture Recognition', clone)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

Hand detected for prediction
1/1 [==============================] - 0s 389ms/step
Hand detected for prediction
1/1 [==============================] - 0s 39ms/step
Hand detected for prediction
1/1 [==============================] - 0s 16ms/step
Hand detected for prediction
1/1 [==============================] - 0s 19ms/step
Hand detected for prediction
1/1 [==============================] - 0s 9ms/step
Hand detected for prediction
1/1 [==============================] - 0s 26ms/step
Hand detected for prediction
1/1 [==============================] - 0s 31ms/step
Hand detected for prediction
1/1 [==============================] - 0s 18ms/step
Hand detected for prediction
1/1 [==============================] - 0s 24ms/step
Hand detected for prediction
1/1 [==============================] - 0s 33ms/step
Hand detected for prediction
1/1 [==============================] - 0s 25ms/step
Hand detected for prediction
1/1 [==============================] - 0s 17ms/step
Hand detected for prediction